# 使用领域（私有）数据微调 ChatGLM3

生成带有 epoch 和 timestamp 的模型文件

## 准备训练数据

In [1]:
!unzip chatglm.zip

Archive:  chatglm.zip
  inflating: chatglm/chatbot_webui.py  
  inflating: chatglm/chatbot_with_memory.ipynb  
  inflating: chatglm/chatglm_inference.ipynb  
   creating: chatglm/data/
  inflating: chatglm/data/raw_data.txt  
  inflating: chatglm/data/zhouyi_dataset_20240118_152413.csv  
  inflating: chatglm/data/zhouyi_dataset_20240118_163659.csv  
  inflating: chatglm/data/zhouyi_dataset_handmade.csv  
  inflating: chatglm/gen_dataset.ipynb  
  inflating: chatglm/qlora_chatglm3.ipynb  
  inflating: chatglm/qlora_chatglm3_timestamp.ipynb  


In [2]:
cp -r chatglm/data ./

## 准备环境 6个 pip install


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00


In [4]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.0 MB/s eta 0:00:00


In [5]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.4 MB/s eta 0:00:00


In [6]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 6.2 MB/s eta 0:00:00


In [7]:
# ValueError                                Traceback (most recent call last)
# <ipython-input-28-2a2a58c6a9f0> in <cell line: 1>()
# ----> 1 trainer = Trainer(
#       2         model=qlora_model,
#       3         args=training_args,
#       4         train_dataset=tokenized_dataset,
#       5         data_collator=data_collator
#
# /usr/local/lib/python3.10/dist-packages/transformers/trainer.py in __init__(self, model, args, data_collator, train_dataset, eval_dataset, tokenizer, model_init, compute_metrics, callbacks, optimizers, preprocess_logits_for_metrics)
#     405                     " inside the model such as adapters using `peft` library and freeze the model weights. Please"
#     406                     " check the examples in https://github.com/huggingface/peft for more details."
# --> 407                 )
#     408             else:
#     409                 raise ValueError(
#
# ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details
# 为了解决以上问题
!pip install transformers==4.33.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [8]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.26.1
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
b

## 导入torch

In [1]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

In [2]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
# train_data_path = 'data/zhouyi_dataset_handmade.csv'    # 训练数据路径
train_data_path = 'data/zhouyi_dataset_20240118_163659.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

## 数据处理

In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 160
    })
})


In [4]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset["train"], num_examples=5)

,content,summary
0,周易中乾卦的解释是什么？,在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。\n\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。
1,比卦在周易中代表了哪些方面？,在周易中，比卦代表着相亲相依的意涵，它是由上卦坎（水）和下卦坤（地）相叠而成。这一卦象预示着长期的吉利和无咎，同时也暗示着不愿臣服的邦国迟迟不来朝会有难。\n\n比卦的核心哲学是：水附大地，地纳河海，象征相亲相依，亲密无间，展示出宽宏无私，精诚团结的道理。因此，在这一卦象中，人们可以得到贵人的提拔，事业可望成功，但需诚实、信任地做事，待人宽厚、正直，主动热情。在经商中，也需真诚交往，遵守商业道德，不可贪心不足或自以为是。\n\n比卦的运势平顺，事业顺利可望成功，可得贵人提拔。经商方面，愿望能够实现且有利润，但需与他人密切合作，讲究商业道德。在婚恋方面，象征着美好姻缘和相亲相爱。在决策中，建议心地善良，待人忠诚、厚道，工作勤恳并善于选择朋友。
2,屯卦的主要讲述内容是什么？,在周易中，屯卦是一个大吉大利的卦象，预示着吉祥和大利。然而，不利于出门，但有利于建国封侯。屯卦由上卦坎（水）下卦震（雷）组成，坎为云，震为雷。预示着云行雷动的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国事。屯卦象征着开始困难，需要毅力和果敢才能获得吉利。身处困境需要多加辛苦努力，排除困难，方可通达，有初难后解之象。因此，对于事业创业而言，应当小心翼翼，勇往直前，灵活机动，可望获得大的成功。但也需注意到仍有困难存在，务必有他人相助，平时应多施恩惠。对于经商，起初多有挫折，必须坚定信念，积极进取，行动果断，若仍无法摆脱困境，则应退守保全，等待机会，再展宏图。对于婚恋，好事多磨，忠贞纯洁，大胆追求，能够成功。屯卦的核心哲学在于，初难后解，需要毅力和坚忍不拔的毅力和锲而不舍的奋斗精神，但也需得到贤德之人的帮助才能摆脱困境。
3,蒙卦涉及哪些哲学思想？,蒙卦是由艮卦（山）下，坎卦（水）上组成的异卦相叠。它代表着通泰，启蒙的意义。在这里，卜者并非是在向幼稚愚昧的人取求，而是幼稚愚昧的人在向卜者求教。第一次卜筮就得到了神灵的指示。然而，如果轻慢不敬地再三卜筮的话，神灵便不会再示警。总的来说，这是一个吉利的卜问。\n\n蒙卦的核心在于山下有泉的形象，寓意着启蒙。君子观此卦象，应当以果敢坚毅的行动来培养自身的品德，像山泉一样果断行动。然而，此卦乃是离宫四世卦，它代表着回还往复、疑惑不前、多忧愁过失，因而属于凶卦。\n\n蒙卦在个人发展、事业经商、求名婚恋等方面的解释不一。在事业方面，表示事业初建，具有启蒙和通达之象，需要勇敢坚毅的行动；而在经商方面，需要务必小心谨慎，树立高尚的商业道德，不可急功近利；求名方面，需要接受良好的基础教育，陶冶情操。整体而言，此卦提示须忍耐待机而动，听取别人意见，方能通达运势。
4,周易中比卦的解释是什么？,在周易中，比卦代表着相亲相依的意涵，它是由上卦坎（水）和下卦坤（地）相叠而成。这一卦象预示着长期的吉利和无咎，同时也暗示着不愿臣服的邦国迟迟不来朝会有难。\n\n比卦的核心哲学是：水附大地，地纳河海，象征相亲相依，亲密无间，展示出宽宏无私，精诚团结的道理。因此，在这一卦象中，人们可以得到贵人的提拔，事业可望成功，但需诚实、信任地做事，待人宽厚、正直，主动热情。在经商中，也需真诚交往，遵守商业道德，不可贪心不足或自以为是。\n\n比卦的运势平顺，事业顺利可望成功，可得贵人提拔。经商方面，愿望能够实现且有利润，但需与他人密切合作，讲究商业道德。在婚恋方面，象征着美好姻缘和相亲相爱。在决策中，建议心地善良，待人忠诚、厚道，工作勤恳并善于选择朋友。


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [8]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False,
    remove_columns=column_names
)

In [9]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

In [10]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}


In [11]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

## 加载模型

In [12]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

In [14]:
target_modules

['query_key_value']

In [15]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [16]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


## QLoRA 微调模型

In [17]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 3
output_dir = f"models/{model_name_or_path}-epoch{train_epochs}-{timestamp}"

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=2,                     # 每个设备的训练批量大小 8 -> 4
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=1,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)


In [19]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

In [20]:
trainer.train()

Step,Training Loss
1,4.770300
2,4.551300
3,4.594200
4,4.726900
5,4.659100
6,4.565300
7,4.516600
8,4.180900
9,3.942400
10,3.857800


TrainOutput(global_step=240, training_loss=0.4205985964649396, metrics={'train_runtime': 157.7382, 'train_samples_per_second': 3.043, 'train_steps_per_second': 1.522, 'total_flos': 2219044483645440.0, 'train_loss': 0.4205985964649396, 'epoch': 3.0})

In [22]:
trainer.model.save_pretrained(output_dir)

# 训练完成后，第一时间保存模型权重文件。

In [35]:
# 使用 trainer.save_model 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
model_dir='models'
model_to_save = trainer.save_model(model_dir)

In [36]:
!ls -lh models

total 15M
-rw-r--r-- 1 root root  588 Feb  6 16:13 adapter_config.json
-rw-r--r-- 1 root root  15M Feb  6 16:13 adapter_model.bin
-rw-r--r-- 1 root root 5.0K Feb  6 16:13 README.md
drwxr-xr-x 4 root root 4.0K Feb  6 15:57 THUDM
-rw-r--r-- 1 root root 4.5K Feb  6 16:13 training_args.bin


In [39]:
# 使用 trainer.save_state 方法保存训练状态
state = trainer.save_state()

In [40]:
print(state)

None


In [43]:
!ls -lh models/THUDM

total 8.0K
drwxr-xr-x  3 root root 4.0K Feb  6 15:46 chatglm3-6b-epoch3-20240206_154651
drwxr-xr-x 27 root root 4.0K Feb  6 16:16 chatglm3-6b-epoch3-20240206_155713


In [44]:
!ls -rlh models/THUDM/*

models/THUDM/chatglm3-6b-epoch3-20240206_155713:
total 16M
-rw-r--r-- 1 root root  29K Feb  6 16:16 trainer_state.json
drwxr-xr-x 3 root root 4.0K Feb  6 15:57 runs
-rw-r--r-- 1 root root 5.0K Feb  6 16:03 README.md
drwxr-xr-x 2 root root 4.0K Feb  6 15:58 checkpoint-90
drwxr-xr-x 2 root root 4.0K Feb  6 15:58 checkpoint-80
drwxr-xr-x 2 root root 4.0K Feb  6 15:58 checkpoint-70
drwxr-xr-x 2 root root 4.0K Feb  6 15:58 checkpoint-60
drwxr-xr-x 2 root root 4.0K Feb  6 15:58 checkpoint-50
drwxr-xr-x 2 root root 4.0K Feb  6 15:58 checkpoint-40
drwxr-xr-x 2 root root 4.0K Feb  6 15:58 checkpoint-30
drwxr-xr-x 2 root root 4.0K Feb  6 16:00 checkpoint-240
drwxr-xr-x 2 root root 4.0K Feb  6 16:00 checkpoint-230
drwxr-xr-x 2 root root 4.0K Feb  6 16:00 checkpoint-220
drwxr-xr-x 2 root root 4.0K Feb  6 16:00 checkpoint-210
drwxr-xr-x 2 root root 4.0K Feb  6 16:00 checkpoint-200
drwxr-xr-x 2 root root 4.0K Feb  6 15:58 checkpoint-20
drwxr-xr-x 2 root root 4.0K Feb  6 15:59 checkpoint-190
drwxr-xr

# 保存到Google Drive


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!ls -lh models

total 4.0K
drwxr-xr-x 4 root root 4.0K Feb  6 15:57 THUDM


In [31]:
%mkdir -p drive/MyDrive/AI/FineTune/homework/week6-chatglm/models

In [32]:
!ls -l drive/MyDrive/AI/FineTune/homework/week6-chatglm/models

total 0


In [34]:
!cp -r models drive/MyDrive/AI/FineTune/homework/week6-chatglm/models

In [37]:
!cp -r models drive/MyDrive/AI/FineTune/homework/week6-chatglm/models

# 其他试错

In [21]:
torch.cuda.empty_cache()

In [23]:
!nvidia-smi

Tue Feb  6 15:49:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              40W / 300W |  15922MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [28]:
!nvidia-smi

Tue Feb  6 15:54:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              40W / 300W |  15922MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [26]:
!nvidia-smi --gpu-reset -i 0

GPU 00000000:00:04.0 is currently in use by another process.

1 device is currently being used by one or more other processes (e.g., Fabric Manager, CUDA application, graphics application such as an X server, or a monitoring application such as another instance of nvidia-smi). Please first kill all processes using this device and all compute applications running in the system.


# 使用 inference Notebook 对比微调前后的效果。
使用领域（私有）数据微调 ChatGLM3

In [41]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [42]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        

In [ ]:
base_model.requires_grad_(False)
base_model.eval()